In [ ]:
# gcloud init
# gcloud auth login
# gcloud services enable \
#   compute.googleapis.com \
#   iam.googleapis.com \
#   iamcredentials.googleapis.com \
#   monitoring.googleapis.com \
#   logging.googleapis.com \
#   notebooks.googleapis.com \
#   aiplatform.googleapis.com \
#   bigquery.googleapis.com \
#   artifactregistry.googleapis.com \
#   cloudbuild.googleapis.com \

In [ ]:
import os
# Step back one directory
os.chdir("..")
# Retrieve and set PROJECT_ID and REGION environment variables.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
REGION = os.environ.get('REGION')
PROJECT_ID, REGION, 

In [ ]:
ARTIFACT_REPOSITORY=os.environ.get('REPOSITORY_NAME')
ARTIFACT_REPOSITORY

In [ ]:
CLOUDBUILD_DIR = os.environ.get('CLOUDBUILD_DIR')
IMAGE_NAME = os.environ.get('IMAGE_NAME')
TAG = os.environ.get('TAG')
CLOUDBUILD_DIR, IMAGE_NAME, TAG

In [ ]:
GCS_BUCKET = f"{PROJECT_ID}-bucket"
!gsutil mb -l $REGION gs://$GCS_BUCKET

In [ ]:
# Create an Artifact Repository using the gcloud CLI.
!gcloud artifacts repositories create $ARTIFACT_REPOSITORY \
--repository-format=docker \
--location=$REGION \
--description="Artifact registry for fastapi test"

In [ ]:
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPOSITORY}/{IMAGE_NAME}:{TAG}"
IMAGE_URI

In [ ]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
  env:
   - 'MODEL_SERVE=PROD'
images: 
- '{IMAGE_URI}'"""

with open(f"{CLOUDBUILD_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

In [ ]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
images: 
- '{IMAGE_URI}'"""

with open(f"{CLOUDBUILD_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

In [ ]:
!gcloud builds submit --timeout=20m --config {CLOUDBUILD_DIR}/cloudbuild.yaml --project {PROJECT_ID}

In [ ]:
!gcloud artifacts repositories list

In [ ]:
!gcloud run services list --project $PROJECT_ID

In [ ]:
url = 'http://localhost:8000/'
url = 'https://fastapi-test-keh6cqmktq-uc.a.run.app/'

In [ ]:
import requests
response = requests.get(url)
print(response.json())
url_pred = f"{url}/predictions"
data = {"instances": ["This is a test"]}
response = requests.post(url_pred, json=data)
print(response.json())